# Retrograde Analyse für das Schach-Endspiel
_Beschreibung_

Bekannte Probleme:
* Nicht optimiert für ein Spielbrett mit Bauern

## Imports
Für das Notebook benötigte Imports
* *chess*: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* *clear_output* aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen
verwendet wird.
* *json*: Wird zum Speichern der Ergebnisse verwendet.

In [13]:
import chess
from IPython.display import clear_output
import json

## Liste der Positionen auf einem Brett (_Square-Names_) bestimmen

Ein Schachbrett ist aufgebaut aus Spalten und Zeilen. Spalten werden durch Buchstaben gekennzeichnet, Zeilen durch
Zahlen. Aus der Kombination einer Spalte (z.B. a) und einer Zahl (z.B. 1) erhält man eine eindeutige
Kennzeichnung für ein Feld (z.B. a1).

Im folgenden Code werden die Buchstaben a bis h mit den Zahlen 1 bis 8 zu diesen Feldnamen konvertiert und in
*all_squares* gespeichert.

In [14]:
column_convert = {
    1 : 'a',
    2 : 'b',
    3 : 'c',
    4 : 'd',
    5 : 'e',
    6 : 'f',
    7 : 'g',
    8 : 'h'
}

all_squares = []
for row in range(1,9):
    for col_num in range(1,9):
        column = column_convert[col_num]
        all_squares.append(column + str(row))
#print(all_squares)

## Einstellen der Figuren, welche Platziert werden sollen
In der folgenden Liste werden alle Figuren aufgeführt, für welche die Endspiel-Datenbank berechnet werden soll.
Jeder eintrag in der Liste ist ein Tupel bestehend aus Figur und Farbe.
Bsp.: ``(chess.KING, chess.WHITE)``

Die Reihenfolge oder Position der Figuren ist für die Berechnung nicht relevant.
Diese wird erst im nächsten Schritt (der Auswertung) benötigt.

In [15]:
figures_to_place = [(chess.BISHOP, chess.WHITE),(chess.BISHOP, chess.WHITE), (chess.KING, chess.WHITE), (chess.KING, chess.BLACK)]

## Die *place\_figure\_everywhere\_on\_every\_board* Hilfsfunktion
Diese Funktion Platziert eine Figur auf allen Feldern jedes Boards in einer übergebenen Liste.

Funktionsargumente:
* *figure*: Die zu platzierende Figur, als Objekt der chess-Library
* *list_of_boards*: Eine Liste mit Boards, auf welchen die Figur platziert werden soll

Nebeneffekte:
* Keine. Es wird mit Kopien der original Objekte gearbeitet.

Algorithmus:
* Über alle Spielbretter in der *list_of_boards* iterieren
* Die bereits verwendeten Spielfelder speichern
* Über alle Spielfelder iterieren
* Wenn das Spielfeld noch nicht benutzt ist:
  * Das Spielbrett kopieren und die Figur auf diesem Feld platzieren
  * Die Kopie der Ergebnisliste anfügen
* Ergebnis zurückgeben

In [16]:
def place_figure_everywhere_on_every_board(figure, list_of_boards):
    result_list = []
    for fen in list_of_boards:
        board = chess.Board(fen)
        cur_piece_map = board.piece_map()
        squares_used = list(cur_piece_map.keys())
        for square in all_squares:
            parsed_square = chess.parse_square(square)
            if parsed_square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(parsed_square, figure)
                result_list.append(tmp_board.fen())
    return result_list

## Die *check_boards* Hilfsfunktion
Diese Funktion überprüft eine Liste von Boards auf Validität und gibt eine Liste valider Boards zurück.

Funktionsargumente:
* *board_list*: Liste mit zu überprüfenden Spielbrettern

Nebeneffekte:
* Keine. Die übergebene Liste wird nicht verändert.

Algorithmus:
Iterieren über alle Boards, überprüfen dieser mittels der *is_valid* Funktion und zurückgeben der validen.

In [17]:
def check_boards(board_list):
    result_list = []
    for fen in board_list:
        board = chess.Board(fen)
        if board.is_valid():
            result_list.append(fen)
    return result_list

## Die Ursprungsmenge S erstellen
Die Liste S enthält alle möglichen Konstellationen der Spielfiguren auf dem Spielbrett.
Hierfür werden zuerst die Figuren auf allen Positionen platziert.
Pro Farbe am Zug (schwarz und weiß) werden somit
$$
\frac{64!}{(64-count(pieces))!}
$$
verschiedene Boards erstellt.
Diese werden anschließend auf Validität überprüft und in einer Liste zusammengeführt.

In [ ]:
empty_board = chess.Board().empty()
white_boards = [empty_board.fen()]
for fig in figures_to_place:
    white_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), white_boards)
    print(len(white_boards))

# All checked boards board.turn = White
S_White = check_boards(white_boards)
print(len(S_White))

empty_board.turn = chess.BLACK
black_boards = [empty_board.fen()]
for fig in figures_to_place:
    black_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), black_boards)
    print(len(black_boards))

# All checked boards board.turn = White
S_Black = check_boards(black_boards)
print(len(S_Black))

S = S_White + S_Black
print(str(len(S)) + " Boards in S")


64
4032
249984


## Gewonnene Spielbretter finden und zu $S_0$ hinzufügen
* Über den Inhalt von $S$ iterieren.
* Status-Informationen von der Bibliothek abfragen und überprüfen, ob ein Spieler gewonnen hat.
* Wenn ja, das Objekt zu $S_0$ hinzufügen.
  Weiter werden die Objekte in ihrer String-Representation zu $S_{0-ASCII}$ hinzugefügt.
* Wenn nicht, das Objekt in $S_{tmp}$ hinzufügen. (Dies ist effizienter als das Objekt aus der Liste $S$ zu entfernen)
* Die ursprüngliche Liste S mit $S_{tmp}$ ersetzen.

In [ ]:
S_tmp = []
S_ASCII = []
S_0 = []
S_0_ASCII = []
S_White_Ascii = []

for fen in S:
    board = chess.Board(fen)
    outcome = board.outcome()
    # If True Game has finished in a certain way
    if outcome is not None:
        # A winner has been determined
        if outcome.winner is not None and board.is_valid():
            S_0.append(board.fen())
            S_0_ASCII.append((board.turn,board.__str__()))
    # Every other board
    else:
        S_tmp.append(board.fen())
        S_ASCII.append((board.turn, board.__str__()))
S = S_tmp

## OneStepAway Funktion
Diese Funktion wählt (aus zwei Listen mit Board-Objekten) die Spielbretter aus der zweiten Liste, welche einen
Spielzug von einem Spielbrett in der ersten Liste entfernt sind.

Funktionsargumente:
* $S_n$: Die Liste mit Board-Objekten, deren Spielbretter sich $n$ Züge vom Sieg entfernt befinden
* $S$: Die Liste, aus welcher die nächste Stufe ausgewählt werden soll
* $S_{white-ASCII}$: Liste der ASCII Representationen aller bereits zugeordneten Spielbretter mit weiß am Zug.
* *iteration_count*: $m$ des $S_m$, welches gerade berechnet wird.

Nebeneffekte:
Keine.
Alle als Parameter übergebenen Listen bleiben unverändert.

Algorithmus:
* Für effizientere Vergleiche für jedes Objekt in $S$ die ASCII-Representation speichern.
* Über die Objekte in $S_n$ iterieren
  * Den Spieler, welcher am Zug ist wechseln (Da um im aktuellen Zustand anzukommen der andere Spieler einen Zug
  gemacht hat)
  * pseudo legale Spielzüge von der Bibliothek berechnen Lassen:
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einem nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren
    * Den Zug ausführen
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Representation überprüfen, ob das modifizierte Spielbrett in $S$ gefunden wird.
    * Wenn es gefunden wird, die ASCII-Representation zu $s_{n+1-ASCII}$ hinzufügen
    * Den Zug rückgängig machen
  * Den Spieler, welcher ursprünglich am Zug war wiederherstellen.
* Über $S$ iterieren
  * Wenn sich das Spielbrett in $S_{n+1-ASCII}$ befindet, wird es $S_{n+1}$ hinzugefügt.
    * Besonderheit für Spielbretter, bei welchen Schwarz am Zug ist: Diese werden nur zu $S_{n+1}$ hinzugefügt, wenn
    alle Züge, welche von dieser Position möglich sind in einem $S_m$ mit $m < n$ enden.
    Für diese Überprüfung wird $S_{white-ASCII}$ verwendet.
  * Wenn sich das Spielbrett nicht in $S_{n+1-ASCII}$ befindet, wird es $S_{tmp}$ hinzugefügt.
* Wurden Positionen mit Weiß am Zug berechnet, diese $S_{white-ASCII}$ hinzufügen.
* $S_{n+1}$, $s_{tmp}$ und $S_{white-ASCII}$ als Tupel zurückgeben.

Ergänzungen zum Algorithmus:
* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden.
Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen können zu einer anderen Figur eingetauscht werden.
Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten
Reihe diese manuell durch einen König ersetzt.


In [ ]:
def one_step_away(s_n, s, s_white_ascii, iteration_count):
    #variables
    s_n1 = []
    s_n1_tmp = []
    s_n1_ascii = []
    s_ascii = []
    s_n_ascii = []
    s_tmp = []

    #create temporary list for comparison
    for fen in s:
        chess_board = chess.Board(fen)
        s_ascii.append((chess_board.turn, chess_board.__str__()))

    for fen in s_n:
        chess_board = chess.Board(fen)
        s_n_ascii.append((chess_board.turn, chess_board.__str__()))

    for i in range(len(s_n)):

        if iteration_count is not None:
            status = "Calculating S" + str(iteration_count) + " - Board " + str(i+1) + " of " + str(len(s_n)) + \
                     " from S" + str(iteration_count-1)
        else:
            status = "Board " + str(i+1) + " of " + str(len(s_n))
        clear_output(wait=True)
        print(status)

        # Get current board and invert the player
        chess_board = chess.Board(s_n[i])
        chess_board.turn = chess_board.turn ^ True

        # Find all Pawns
        pawn_at = []
        for pawn in chess_board.pieces(chess.PAWN, True):
            pawn_at.append(chess.square_name(pawn))

        # try moves and check if they remain in S
        for pLMove in chess_board.pseudo_legal_moves:
            if chess.square_name(pLMove.from_square) not in pawn_at:
                chess_board.push(pLMove)
                if not chess_board.is_valid():
                    chess_board.pop()
                    continue
                # If the new board is found in S, it can be reached in one step
                if (chess_board.turn ^ True, chess_board.__str__()) in s_ascii:
                    s_n1_ascii.append((chess_board.turn ^ True, chess_board.__str__()))
                chess_board.pop()
        '''
        * As pawns only move forward their move cannot cannot be reversed through the pseudo legal moves
        * to mitigate this the pawn is moved back one row manually
        '''
        for pawn in chess_board.pieces(chess.PAWN, True):
            if chess_board.piece_at(pawn - 9) is None:
                chess_board.remove_piece_at(pawn)
                chess_board.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
                print(chess_board)
                if chess_board.is_valid():
                    if (chess_board.turn ^ True, chess_board.__str__()) in s_ascii:
                        s_n1_ascii.append((chess_board.turn ^ True, chess_board.__str__()))
                chess_board.remove_piece_at(pawn - 9)
                chess_board.set_piece_at(pawn, chess.Piece.from_symbol('P'))
        '''
        * As pawns can be exchanged to other figures and this cannot be undone this exchange can also not be reversed
        * through the pseudo legal moves.
        * To mitigate this a pawn will be placed in the top row in the spot of a queen
        '''
        queen = check_top_row(chess_board)
        if queen:
            for square in queen:
                chess_board.remove_piece_at(square)
                chess_board.set_piece_at(square, chess.Piece.from_symbol('P'))
                if chess_board.is_valid():
                    if (chess_board.turn ^ True, chess_board.__str__()) in s_ascii:
                        s_n1_ascii.append((chess_board.turn ^ True, chess_board.__str__()))
                chess_board.remove_piece_at(square)
                chess_board.set_piece_at(square, chess.Piece.from_symbol('Q'))

        # Restore the original state of the board
        chess_board.turn = chess_board.turn ^ True

    clear_output(wait=True)
    if iteration_count is not None:
        print("Calculating S" + str(iteration_count) + " - Mapping ASCII boards to board objects")
    else:
        print("Mapping ASCII boards to board objects")
    #create return lists
    for fen in s:
        chess_board = chess.Board(fen)
        if(chess_board.turn, chess_board.__str__()) in s_n1_ascii:
            s_n1.append(chess_board.fen())
        else:
            s_tmp.append(chess_board.fen())


    clear_output(wait=True)
    if iteration_count is not None:
        print("Calculating S" + str(iteration_count) + " - Checking Black Moves for determinism")
    else:
        print("Checking Black Moves for determinism")

    '''
    * We can choose every turn white does, but not for black.
    * If one particular move from black brings the board from S_n to S_n-1 it is not guaranteed that every move
    * black can do does so as well.
    * To mitigate this only boards where every move ends up in a lower denomination of S will be added to the currently
    * calculated one.
    * For efficiency an ascii representation is carried through all calculations.
    '''
    # Only needed for Black-Moves
    for fen in s_n1:
        chess_board = chess.Board(fen)
        include = True
        if not chess_board.turn:
            for move in chess_board.legal_moves:
                chess_board.push(move)
                str_rep = (chess_board.turn, chess_board.__str__())
                chess_board.pop()
                if str_rep not in s_white_ascii:
                    include = False
        if include:
            s_n1_tmp.append(chess_board.fen())
        else:
            s_tmp.append(chess_board.fen())
    s_n1 = s_n1_tmp

    # Only needed for White-Moves
    if iteration_count % 2 == 1:
        s_white_ascii = s_white_ascii + s_n1_ascii


    clear_output(wait=True)
    print("Done with S" + str(iteration_count) + " - " + str(len(s_tmp)) + " Boards remaining in S")



    return s_n1, s_tmp, s_white_ascii

## Die *check\_top\_row* Hilfsfunktion
Diese Funktion überprüft, ob sich in der obersten Reihe eines Boards eine Königin befindet und gibt ggf.
die Quadrat-Nummer(n) zurück.

Funktionsargumente:
* *board*: Das zu überprüfende Spielbrett als Objekt der chess-Library

Nebeneffekte:
* Keine. Das übergebene Objekt wird nicht verändert.

Algorithmus:
Es wird jedes Feld der obersten Reihe in einer überprüft und die Nummern der Felder mit einer Königin zurückgegeben.

In [ ]:
def check_top_row(board):
    return_list = []
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return_list.append(i)

    if len(return_list) > 0:
        return return_list
    else:
        return False

## In einer Schleife die Liste mit $S_n$ berechnen
Solange Boards einem $n$ zugeordnet werden, wird die Schleife weitergeführt. Sie wird abgebrochen,
wenn $S_{n+1}$ leer ist.

In [ ]:
S_n_sequence = [S_0]
while True:
    S_n_new, S, S_White_Ascii = one_step_away(S_n_sequence[-1], S, S_White_Ascii, len(S_n_sequence))
    if not S_n_new: #an empty list is false
        break
    S_n_sequence.append(S_n_new)

print("Done")
print(str(len(S_n_sequence)) + " S-Lists calculated")
print(str(len(S)) + " Boards could not be matched into an n.")

## Ergebnis in Datei schreiben
Nach erfolgreicher Berechnung werden die FENs der Boards in eine Datei geschrieben, um eine spätere Auswertung
zu ermöglichen.

In [ ]:
S_n_seq_ascii = []
S_ascii = []

f = open("S_n_seq.json", "w")
f.write(json.dumps(S_n_sequence))
f.close()

f = open("S_rem.json", "w")
f.write(json.dumps(S))
f.close()